In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标网址的基础部分
base_url = "https://www.bu.edu/classrooms/find-a-classroom/page/{}/?cts_address&cts_capacity_lfa&cts_capacity&cts_filter_submit=Search"

# 打开一个CSV文件来保存结果
with open('../data/classroom_links.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入CSV文件的标题行
    writer.writerow(['Link'])

    page = 1  # 从第一页开始
    while True:
        # 获取每页的URL
        url = base_url.format(page)
        response = requests.get(url)
        html_content = response.content

        # 使用BeautifulSoup解析网页
        soup = BeautifulSoup(html_content, 'html.parser')

        # 检查是否存在<p class="cts-error">No classrooms found.</p>
        error_message = soup.find('p', class_='cts-error')
        if error_message:
            print(f"在第 {page} 页找不到教室，停止爬取。")
            break

        # 查找<ul class="cts-results-list">中的<li>
        results_list = soup.find('ul', class_='cts-results-list')
        if results_list:
            li_items = results_list.find_all('li')

            # 提取每个<li>中的<a>标签中的href链接
            for li in li_items:
                link_tag = li.find('a', class_='cts-button cts-button-primary')
                if link_tag and link_tag.get('href'):
                    classroom_link = link_tag['href']
                    # 写入CSV文件
                    writer.writerow([classroom_link])

        # 输出当前爬取的页码信息
        print(f"已完成第 {page} 页的爬取。")
        page += 1  # 移动到下一页

print("所有页面的教室链接已经成功保存到 'classroom_links.csv' 文件中。")

已完成第 1 页的爬取。
已完成第 2 页的爬取。
已完成第 3 页的爬取。
已完成第 4 页的爬取。
已完成第 5 页的爬取。
已完成第 6 页的爬取。
已完成第 7 页的爬取。
已完成第 8 页的爬取。
已完成第 9 页的爬取。
已完成第 10 页的爬取。
已完成第 11 页的爬取。
已完成第 12 页的爬取。
已完成第 13 页的爬取。
已完成第 14 页的爬取。
已完成第 15 页的爬取。
已完成第 16 页的爬取。
已完成第 17 页的爬取。
已完成第 18 页的爬取。
已完成第 19 页的爬取。
已完成第 20 页的爬取。
已完成第 21 页的爬取。
已完成第 22 页的爬取。
已完成第 23 页的爬取。
已完成第 24 页的爬取。
已完成第 25 页的爬取。
已完成第 26 页的爬取。
已完成第 27 页的爬取。
已完成第 28 页的爬取。
已完成第 29 页的爬取。
已完成第 30 页的爬取。
已完成第 31 页的爬取。
已完成第 32 页的爬取。
已完成第 33 页的爬取。
在第 34 页找不到教室，停止爬取。
所有页面的教室链接已经成功保存到 'classroom_links.csv' 文件中。


In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import json

# 从CSV中读取所有链接
urls = []
with open('../data/classroom_links.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 跳过标题行
    for row in reader:
        urls.append(row[0])

# 用于存储所有页面信息的列表
classroom_data = []

# 遍历每个URL并获取页面信息
for url in urls:
    response = requests.get(url)
    html_content = response.content

    # 使用BeautifulSoup解析网页
    soup = BeautifulSoup(html_content, 'html.parser')

    # 1. 提取Name属性，来自<div class="content-panel">中的<h1>元素
    name = soup.find('div', class_='content-panel').find('h1').get_text().strip()

    # 2. 查找<div class="cts-detail-container">并提取其中的信息
    detail_container = soup.find('div', class_='cts-detail-container')
    details = detail_container.find_all('div', class_='cts-detail-details')

    classroom_info = {
        "Name": name,
        "Details": {},
        "AdditionalInfo": {}
    }

    # 处理第一个 cts-detail-details，提取每个<h4>和对应的<ul>
    first_detail = details[0] if len(details) > 0 else None
    if first_detail:
        h4_elements = first_detail.find_all('h4')

        for h4 in h4_elements:
            h4_title = h4.get_text().strip()

            # 找到<h4>后面的第一个<ul>
            ul_list = h4.find_next('ul', class_='cts-detail-list')
            if ul_list:
                li_items = ul_list.find_all('li')
                li_texts = []
                for li in li_items:
                    li_text = li.get_text(strip=True).replace(li.find('span').get_text(strip=True), "").strip()
                    li_texts.append(li_text)

                classroom_info["Details"][h4_title] = li_texts

    # 处理第二个 cts-detail-details，提取6个<li>中的meta-name和meta-value
    second_detail = details[1] if len(details) > 1 else None
    if second_detail:
        li_items = second_detail.find_all('li')
        for li in li_items:
            meta_name = li.find('span', class_='meta-name').get_text(strip=True)
            meta_value = li.find('span', class_='meta-value').get_text(strip=True)
            classroom_info["AdditionalInfo"][meta_name] = meta_value

    # 将该教室的信息加入列表
    classroom_data.append(classroom_info)

# 将数据保存为JSON文件
with open('../data/classroom_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(classroom_data, json_file, ensure_ascii=False, indent=4)

print("教室信息已成功保存到 'classroom_data.json' 文件中。")


教室信息已成功保存到 'classroom_data.json' 文件中。
